In [2]:
import pandas as pd
import time
%matplotlib inline
from datetime import datetime
import scipy.spatial
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import HuberRegressor 
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor

In [16]:
postulaciones = pd.read_csv('Data/Postulaciones/Postulaciones-merge.csv')

In [17]:
postulaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8311264 entries, 0 to 8311263
Data columns (total 3 columns):
idaviso             int64
idpostulante        object
fechapostulacion    object
dtypes: int64(1), object(2)
memory usage: 190.2+ MB


In [22]:
postulantes_educacion = pd.read_csv('Data/Postulantes-Educacion/postulantes_educacion-merge.csv')

In [23]:
postulantes_educacion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 447909 entries, 0 to 447908
Data columns (total 4 columns):
idpostulante       447909 non-null object
nombre             447909 non-null object
estado             447909 non-null object
nombre_numerico    447909 non-null int64
dtypes: int64(1), object(3)
memory usage: 13.7+ MB


In [7]:
postulantes_genero_edad = pd.read_csv('Data/Postulantes-Genero-Edad/postulantes_genero_y_edad-merge.csv')

In [8]:
postulantes_genero_edad.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 504407 entries, 0 to 504406
Data columns (total 3 columns):
idpostulante       504407 non-null object
fechanacimiento    477884 non-null object
sexo               504407 non-null object
dtypes: object(3)
memory usage: 11.5+ MB


In [20]:
avisos_online = pd.read_csv('Data/Avisos-Online/Avisos-Online-merge.csv')

In [21]:
avisos_online.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9886 entries, 0 to 9885
Data columns (total 1 columns):
idaviso    9886 non-null int64
dtypes: int64(1)
memory usage: 77.3 KB


In [18]:
avisos_detalle = pd.read_csv('Data/Avisos-Detalle/Avisos-Detalle-merge.csv')

In [19]:
avisos_detalle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45969 entries, 0 to 45968
Data columns (total 11 columns):
idaviso                 45969 non-null int64
idpais                  45969 non-null int64
titulo                  45969 non-null object
descripcion             45969 non-null object
nombre_zona             45969 non-null object
ciudad                  225 non-null object
mapacalle               3444 non-null object
tipo_de_trabajo         45969 non-null object
nivel_laboral           45634 non-null object
nombre_area             45969 non-null object
denominacion_empresa    45955 non-null object
dtypes: int64(2), object(9)
memory usage: 3.9+ MB
